### Load Python Packages

In [1]:
!pip install twelvelabs

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 57.3 MB/s eta 0:00:00
  Attempting uninstall: pydantic-core
    Found existing installation: pydantic_core 2.18.4
    Uninstalling pydantic_core-2.18.4:
      Successfully uninstalled pydantic_core-2.18.4
  Attempting uninstall: pydantic
    Found existing installation: pydantic 1.10.17
    Uninstalling pydantic-1.10.17:
      Successfully uninstalled pydantic-1.10.17
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-community 0.2.12 requires langchain<0.3.0,>=0.2.13, but you have langchain 0.2.5 which is incompatible.


### Load API key & Initialize TwelveLab Client

In [19]:
import json
from twelvelabs import TwelveLabs
from twelvelabs import APIStatusError
from twelvelabs.models.task import Task
from pathlib import Path
from IPython.display import Video

def load_api_key(file_path='config.json', key_name='api_key'):
    try:
        with open(file_path, 'r') as file:
            config = json.load(file)
        return config.get(key_name)
    except FileNotFoundError:
        print(f"Error: The file {file_path} was not found.")
    except json.JSONDecodeError:
        print(f"Error: The file {file_path} is not a valid JSON file.")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
    return None

# Usage example:
api_key = load_api_key(file_path="access.json", key_name="API_KEY")

client = TwelveLabs(api_key=api_key)

### Setup Video Index

In [8]:
from typing import List, Optional

def get_or_create_index(client, index_name: str) -> Optional[str]:
    """
    Get an existing index or create a new one if it doesn't exist.

    Args:
        client: The client object to interact with the index API.
        index_name (str): The name of the index to get or create.
        engines (List[dict]): A list of engine configurations for the new index.

    Returns:
        Optional[str]: The ID of the existing or newly created index, or None if an error occurred.
    """
    try:
        # Get a list of existing indexes
        indexes = client.index.list(page=1)

        # Check if an index with the desired name already exists
        for index in indexes:
            if index.name == index_name:
                print(f"Using existing index '{index.name}' (id={index.id})")
                return index.id

        # Example usage
        engines = [
            {
                "name": "marengo2.6",
                "options": ["visual", "conversation", "text_in_video"],
            },
            {
                "name": "pegasus1.1",
                "options": ["visual", "conversation"],
            },
        ]
        # If the index doesn't exist, create a new one
        index_obj = client.index.create(
            name=index_name,
            engines=engines,
        )
        print(f"Created new index '{index_obj.name}' (id={index_obj.id})")
        return index_obj.id

    except APIStatusError as e:
        print('API Status Error, 4xx or 5xx')
        print(e)
    except Exception as e:
        print(e)

    return None

### Create Video Index

In [11]:
index_id = get_or_create_index(client, "My Index (Default)")
if index_id:
    print(f"Index ID: {index_id}")
else:
    print("Failed to get or create index.")

Using existing index 'My Index (Default)' (id=66c69fe2d057881c8deee0fc)
Index ID: 66c69fe2d057881c8deee0fc


### Download sample video

In [17]:
# Save information about the video in a dictionary as we progress through the notebook
video = {}
video["mp4_file"] = 'Netflix_Open_Content_Meridian.mp4'
video["video_dir"] = Path(video["mp4_file"]).stem
video["url"] = f"https://dx2y1cac29mt3.cloudfront.net/mp4/netflix/{video['mp4_file']}"

!curl {video["url"]} -o {video["mp4_file"]}

Video(url=video["url"], width=640, height=360, html_attributes="controls muted autoplay")

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 47.1M  100 47.1M    0     0   115M      0 --:--:-- --:--:-- --:--:--  115M


### Create an Index of the Video

In [21]:
task = client.task.create(
  index_id=index_id,
  file=video["mp4_file"],
  # transcription_file="<YOUR_TRANSCRIPTION_FILE>"
)
print(f"Task id={task.id}")

# Utility function to print the status of a video indexing task
def on_task_update(task: Task):
      print(f"  Status={task.status}")

task.wait_for_done(sleep_interval=5, callback=on_task_update)

if task.status != "ready":
  raise RuntimeError(f"Indexing failed with status {task.status}")
print(f"The unique identifer of your video is {task.video_id}.")

Task id=66eebe5a5bfc453560996ccf
  Status=pending
  Status=pending
  Status=pending
  Status=pending
  Status=indexing
  Status=indexing
  Status=indexing
  Status=indexing
  Status=indexing
  Status=indexing
  Status=indexing
  Status=indexing
  Status=indexing
  Status=indexing
  Status=indexing
  Status=indexing
  Status=indexing
  Status=indexing
  Status=indexing
  Status=indexing
  Status=indexing
  Status=ready
The unique identifer of your video is 66eebe5bd4f452b062c63f01.


### Generate title, topic, and hashtag


In [24]:
%time
gist = client.generate.gist(
  video_id=task.video_id,
  types=["title", "topic", "hashtag"]
)
print(f"Title={gist.title}\nTopic={gist.topics}\nHashtags={gist.hashtags}")

CPU times: user 3 μs, sys: 0 ns, total: 3 μs
Wall time: 5.96 μs
Title=Mysterious Disappearances and Dark Secrets in 1947 Hollywood
Topic=root=['Hollywood Noir Mystery']
Hashtags=root=['LosAngeles', '1947', 'Hollywood', 'Mystery', 'Detective', 'Noir', 'Vintage']


### Generate Summary

In [26]:
%time
res = client.generate.summarize(
  video_id=task.video_id,
  type="summary"
)
print(f"Summary= {res.summary}")

CPU times: user 2 μs, sys: 0 ns, total: 2 μs
Wall time: 6.2 μs
Summary= The video takes viewers on a journey through Los Angeles in 1947, beginning with a bustling city street scene that highlights iconic landmarks such as the Hollywood sign, Warner Bros. Theatre, Roosevelt Hotel, and Chinese Theatre. It then shifts focus to a law office where a man named Samuel Goldwyn discusses a mysterious case involving several missing men. The dialogue reveals that the men had no apparent connections to criminal activity but were all divorced. The conversation hints at a potential link to a notorious figure, Mickey Cohen, and introduces a witness who saw strange occurrences at the site of the disappearances.
As the narrative unfolds, the witness recounts seeing a man on a rock at El Matador beach, followed by a sudden and intense storm. In the midst of this chaos, the man disappears, and a woman in a white dress with alabaster skin appears, staring directly at the witness. This eerie description s

### Generate Chapters

In [27]:
res = client.generate.summarize(
    video_id=task.video_id,
    type="chapter",
)
for chapter in res.chapters:
    print(
        f"""Chapter {chapter.chapter_number},
start={chapter.start},
end={chapter.end}
Title: {chapter.chapter_title}
Summary: {chapter.chapter_summary}
"""
    )

Chapter 0,
start=0,
end=25
Title: Introduction to 1947 Los Angeles
Summary: The video opens with a bustling city street in Los Angeles, showcasing various landmarks and historic buildings.

Chapter 1,
start=25,
end=68
Title: The Law Office
Summary: A man named Samuel Goldwyn is seen in his law office, discussing a mysterious case with a client.

Chapter 2,
start=69,
end=180
Title: The Mysterious Disappearances
Summary: The conversation continues, revealing details about a series of mysterious disappearances and a witness account.

Chapter 3,
start=180,
end=240
Title: The Witness Account
Summary: The witness describes a strange event involving a woman in a white dress at El Matador.

Chapter 4,
start=240,
end=331
Title: Investigation at El Matador
Summary: Captain Foster investigates the El Matador location, looking for Detective Sullivan.

Chapter 5,
start=331,
end=540
Title: The Stormy Encounter
Summary: A man and a woman navigate a stormy beach and a cave, leading to a tense confront

### Generate highlights

In [29]:
res = client.generate.summarize(
    video_id=task.video_id,
    type="highlight"
)
for highlight in res.highlights:
    print(f"Highlight: {highlight.highlight}, start: {highlight.start}, end: {highlight.end}")

Highlight: Discussion about Disappearances, start: 26, end: 67
Highlight: Witness Account, start: 69, end: 124
Highlight: Mysterious Woman, start: 153, end: 182
Highlight: Recounting the Witness's Story, start: 210, end: 259
Highlight: Captain Foster at El Matador, start: 311, end: 331
Highlight: Thank You, start: 440, end: 442
Highlight: LAPD Confrontation, start: 529, end: 540
Highlight: LAPD Confrontation Continued, start: 540, end: 550


### Generate Open-ended text

In [34]:
custom_prompt = """
please detect the end credits of a video and generating the start and end timestamps in JSON format.

To identify end credits, use the following criteria:

- Scrolling or static text appearing at the end of the video
- Names and roles of cast and crew members
- Usually appears after the main content has concluded
- Often accompanied by background music or a black background

Generate a JSON object with the following structure: { "endCredits": { "start": "MM:SS", "end": "MM:SS" } }. Skip any explanation.
"""

res = client.generate.text(
  video_id=task.video_id,
  prompt=custom_prompt
)
print(f"{res.data}")

```json
{
  "endCredits": {
    "start": "02:56",
    "end": "03:00"
  }
}
```
